In [ ]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.analysis as analysis
from TRITON_SWMM_toolkit.process_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.process_simulation as ps
import TRITON_SWMM_toolkit.processing_analysis as pe
from TRITON_SWMM_toolkit.process_simulation import *
import TRITON_SWMM_toolkit.plot_utils as plot_utils
import TRITON_SWMM_toolkit.utils as utils 
import TRITON_SWMM_toolkit.run_simulation as run
import TRITON_SWMM_toolkit.sensitivity_analysis as snst
from TRITON_SWMM_toolkit.config import analysis_config
from TRITON_SWMM_toolkit.examples import GetTS_TestCases as tst
from TRITON_SWMM_toolkit.scenario import TRITONSWMM_scenario
from importlib import reload
reload(examples)
reload(analysis)
reload(ps)
reload(pe)
reload(plot_utils)
reload(utils)
reload(snst)
rebuild_case = False

# single sim test case

In [ ]:
testcase = tst.retreive_norfolk_single_sim_test_case(
        start_from_scratch=rebuild_case
    )
self = testcase.system.analysis
analysis = self
assert analysis.compilation_successful

In [ ]:
analysis.run_prepare_scenarios_serially(
        overwrite_scenarios=True, rerun_swmm_hydro_if_outputs_exist=True
    )

In [ ]:
scen = TRITONSWMM_scenario(0, analysis)
scen.log.scenario_creation_complete.get()
not scen.log.scenario_creation_complete.get()

In [ ]:
# def test_process_sim():
single_sim_single_core = tst.retreive_norfolk_single_sim_test_case()
analysis = single_sim_single_core.system.analysis
analysis.process_all_sim_timeseries_serially()
success_processing = (
    analysis.log.all_TRITON_timeseries_processed.get()
    and analysis.log.all_SWMM_timeseries_processed.get()
)
if not success_processing:
    analysis.print_logfile_for_scenario(0)
    print(f"Processing TRITON and SWMM time series failed.")
success_clearing = (
    analysis.log.all_raw_TRITON_outputs_cleared.get()
    and analysis.log.all_raw_SWMM_outputs_cleared.get()
)
if not success_clearing:
    analysis.print_logfile_for_scenario(0)
    print(f"Clearning raw outputs failed.")

# multi-sim testcase

In [ ]:
testcase = tst.retreive_norfolk_multi_sim_test_case(start_from_scratch=False)
analysis = testcase.system.analysis
self = analysis.process
assert analysis.log.TRITONSWMM_compiled_successfully.get()

In [ ]:
prepare_scenario_launchers = analysis.retrieve_prepare_scenario_launchers(
        overwrite_scenario=True, verbose=True
    )
analysis.run_python_functions_concurrently(prepare_scenario_launchers)

In [ ]:
analysis.log.all_scenarios_created.get()

In [ ]:
launch_functions = analysis._create_launchable_sims(
        pickup_where_leftoff=False, verbose=True
    )
analysis.run_simulations_concurrently_on_desktop(launch_functions, verbose=True)

# sensitivity analysis

In [2]:
nrflk_cpu_sensitivity = tst.retreive_norfolk_cpu_config_sensitivity_case(
        start_from_scratch=True
    )

In [3]:
testcase = tst.retreive_norfolk_cpu_config_sensitivity_case(start_from_scratch=False)
analysis = testcase.system.analysis

In [4]:
analysis.sensitivity.compile_TRITON_SWMM_for_sensitivity_analysis()
assert analysis.sensitivity.compilation_successful == True

In [5]:
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True

In [6]:
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=False)
# assert analysis.log.all_sims_run.get() == True